# 1: MTC MNL Mode Choice

In [8]:
import pandas as pd
import larch.numba as lx

In [9]:
# TEST
pd.set_option("display.max_columns", 999)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 3)
import larch
larch._doctest_mode_ = True
from pytest import approx

This example is a mode choice model built using the MTC example dataset.
First we create the Dataset and Model objects:

In [10]:
d = lx.examples.MTC(format='dataset')
d

<xarray.Dataset>
Dimensions:    (caseid: 5029, altid: 6)
Coordinates:
  * caseid     (caseid) int64 1 2 3 4 5 6 7 ... 5024 5025 5026 5027 5028 5029
  * altid      (altid) int64 1 2 3 4 5 6
    alt_names  (altid) <U7 'DA' 'SR2' 'SR3+' 'Transit' 'Bike' 'Walk'
Data variables: (12/38)
    hhid       (caseid) int64 2 3 5 6 8 8 12 ... 9429 9430 9433 9434 9436 9438
    perid      (caseid) int64 1 1 1 1 2 3 2 2 1 1 2 2 ... 2 1 1 2 1 1 1 1 1 1 1
    numalts    (caseid) int64 2 2 2 2 2 2 2 2 2 2 2 2 ... 2 2 2 2 2 2 2 2 2 2 2
    dist       (caseid) float64 7.69 11.62 4.1 14.58 ... 4.34 12.06 2.95 0.73
    wkzone     (caseid) int64 664 738 696 665 679 ... 1002 1012 1009 1030 1021
    hmzone     (caseid) int64 726 9 667 8 704 665 ... 978 1023 1019 1030 1021
    ...         ...
    ivtt       (caseid, altid) float64 13.38 18.38 20.38 25.9 ... 1.59 6.55 0.0
    ovtt       (caseid, altid) float64 2.0 2.0 2.0 15.2 2.0 ... 4.5 16.0 4.5 0.0
    tottime    (caseid, altid) float64 15.38 20.38 22.38 ... 17.59 11.05 19.1
    totcost    (caseid, altid) float64 70.63 35.32 20.18 115.6 ... 75.0 0.0 0.0
    altnum     (caseid, altid) int64 1 2 3 4 5 0 1 2 3 4 ... 3 4 0 6 1 2 3 4 5 6
    avail      (caseid, altid) int8 1 1 1 1 1 0 1 1 1 1 ... 1 1 0 1 1 1 1 1 1 1
Attributes:
    _caseid_:  caseid
    _altid_:   altid

In [11]:
m = lx.Model(d)

Then we can build up the utility function.  We'll use some :ref:`idco` data first, using
the `Model.utility.co` attribute.  This attribute is a dict-like object, to which
we can assign :class:`LinearFunction` objects for each alternative code.

In [12]:
from larch import P, X, PX
m.utility_co[2] = P("ASC_SR2")  + P("hhinc#2") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we'll use some idca data, with the `utility_ca` attribute. This attribute
is only a single :class:`LinearFunction` that is applied across all alternatives
using :ref:`idca` data.  Because the data is structured to vary across alternatives,
the parameters (and thus the structure of the :class:`LinearFunction`) does not need
to vary across alternatives.

In [13]:
m.utility_ca = PX("tottime") + PX("totcost")

Lastly, we need to identify :ref:`idca` data that gives the availability for each
alternative, as well as the number of times each alternative is chosen. (In traditional
discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)

In [14]:
m.availability_var = 'avail'
m.choice_ca_var = 'chose'

And let's give our model a descriptive title.

In [15]:
m.title = "MTC Example 1 (Simple MNL)"

We can view a summary of the choices and alternative 
availabilities to make sure the model is set up 
correctly.

In [16]:
m.choice_avail_summary()

,name,chosen,available
altid,,,
1,DA,3637,4755
2,SR2,517,5029
3,SR3+,161,5029
4,Transit,498,4003
5,Bike,50,1738
6,Walk,166,1479
< Total All Alternatives >,,5029,


In [17]:
# TEST
s = '''            name  chosen available
altid                                    
1                    DA    3637      4755
2                   SR2     517      5029
3                  SR3+     161      5029
4               Transit     498      4003
5                  Bike      50      1738
6                  Walk     166      1479
< Total All Alternatives > 5029          
'''
import re
mash = lambda x: re.sub('\s+', ' ', x).strip()
assert mash(s) == mash(str(m.choice_avail_summary()))

Having created this model, we can then estimate it:

In [18]:
# TEST
assert dict(m.required_data()) == {
    'ca': ['totcost', 'tottime'],
    'co': ['hhinc'],
    'choice_ca': 'chose',
    'avail_ca': 'avail',
}
assert m.loglike() == approx(-7309.600971749634)

In [19]:
m.maximize_loglike()

┣          loglike: -3626.18625551293
┣                x: ASC_BIKE   -2.376e+00
┃                   ASC_SR2    -2.178e+00
┃                   ASC_SR3P   -3.725e+00
┃                   ASC_TRAN   -6.709e-01
┃                   ASC_WALK   -2.068e-01
┃                   hhinc#2    -2.170e-03
┃                   hhinc#3     3.577e-04
┃                   hhinc#4    -5.286e-03
┃                   hhinc#5    -1.281e-02
┃                   hhinc#6    -9.686e-03
┃                   totcost    -4.920e-03
┃                   tottime    -5.134e-02
┃                   dtype: float64
┣        tolerance: 2.0071751679178634e-06
┣            steps: array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
┣          message: 'Optimization terminated successfully.'
┣     elapsed_time: datetime.timedelta(microseconds=183565)
┣           method: 'bhhh'
┣          n_cases: 5029
┣ iteration_number: 0
┣          logloss: 0.7210551313408093
┣ __verbose_repr__: True

In [20]:
# TEST
result = _
assert result.loglike == approx(-3626.18625551293)
assert result.logloss == approx(0.7210551313408093)
assert result.message == 'Optimization terminated successfully.'

In [21]:
m.calculate_parameter_covariance()

In [22]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_BIKE,-2.38,0.305,-7.80,***,0.00
ASC_SR2,-2.18,0.105,-20.81,***,0.00
ASC_SR3P,-3.73,0.178,-20.96,***,0.00
ASC_TRAN,-0.671,0.133,-5.06,***,0.00
ASC_WALK,-0.207,0.194,-1.07,,0.00
hhinc#2,-0.00217,0.00155,-1.40,,0.00
hhinc#3,0.000358,0.00254,0.14,,0.00
hhinc#4,-0.00529,0.00183,-2.89,**,0.00
hhinc#5,-0.0128,0.00532,-2.41,*,0.00
hhinc#6,-0.00969,0.00303,-3.19,**,0.00


In [ ]:
# TEST
summary = _
assert (summary.data.to_markdown()) == '''
|          |     Value |   Std Err |   t Stat | Signif   |   Null Value |
|:---------|----------:|----------:|---------:|:---------|-------------:|
| ASC_BIKE | -2.38     |  0.305    |    -7.8  | ***      |            0 |
| ASC_SR2  | -2.18     |  0.105    |   -20.81 | ***      |            0 |
| ASC_SR3P | -3.73     |  0.178    |   -20.96 | ***      |            0 |
| ASC_TRAN | -0.671    |  0.133    |    -5.06 | ***      |            0 |
| ASC_WALK | -0.207    |  0.194    |    -1.07 |          |            0 |
| hhinc#2  | -0.00217  |  0.00155  |    -1.4  |          |            0 |
| hhinc#3  |  0.000358 |  0.00254  |     0.14 |          |            0 |
| hhinc#4  | -0.00529  |  0.00183  |    -2.89 | **       |            0 |
| hhinc#5  | -0.0128   |  0.00532  |    -2.41 | *        |            0 |
| hhinc#6  | -0.00969  |  0.00303  |    -3.19 | **       |            0 |
| totcost  | -0.00492  |  0.000239 |   -20.6  | ***      |            0 |
| tottime  | -0.0513   |  0.0031   |   -16.57 | ***      |            0 |
'''[1:-1]

It is a little tough to read this report because the parameters can show up 
in pretty much any order, as they are not sorted
when they are automatically discovered by Larch.
We can use the reorder method to fix this:

In [23]:
m.ordering = (
    ("LOS", "totcost", "tottime", ),
    ("ASCs", "ASC.*", ),
    ("Income", "hhinc.*", ),
)

In [24]:
m.parameter_summary()

In [ ]:
# TEST
summary2 = _
assert summary2.data.to_markdown() == '''
|                       |     Value |   Std Err |   t Stat | Signif   |   Null Value |
|:----------------------|----------:|----------:|---------:|:---------|-------------:|
| ('LOS', 'totcost')    | -0.00492  |  0.000239 |   -20.6  | ***      |            0 |
| ('LOS', 'tottime')    | -0.0513   |  0.0031   |   -16.57 | ***      |            0 |
| ('ASCs', 'ASC_BIKE')  | -2.38     |  0.305    |    -7.8  | ***      |            0 |
| ('ASCs', 'ASC_SR2')   | -2.18     |  0.105    |   -20.81 | ***      |            0 |
| ('ASCs', 'ASC_SR3P')  | -3.73     |  0.178    |   -20.96 | ***      |            0 |
| ('ASCs', 'ASC_TRAN')  | -0.671    |  0.133    |    -5.06 | ***      |            0 |
| ('ASCs', 'ASC_WALK')  | -0.207    |  0.194    |    -1.07 |          |            0 |
| ('Income', 'hhinc#2') | -0.00217  |  0.00155  |    -1.4  |          |            0 |
| ('Income', 'hhinc#3') |  0.000358 |  0.00254  |     0.14 |          |            0 |
| ('Income', 'hhinc#4') | -0.00529  |  0.00183  |    -2.89 | **       |            0 |
| ('Income', 'hhinc#5') | -0.0128   |  0.00532  |    -2.41 | *        |            0 |
| ('Income', 'hhinc#6') | -0.00969  |  0.00303  |    -3.19 | **       |            0 |
'''[1:-1]


In [25]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

In [26]:
# TEST
estats = _
from xmle.elem import Elem
assert isinstance(estats, Elem)
assert m._cached_loglike_best == approx(-3626.18625551293)
assert m._cached_loglike_null == approx(-7309.600971749634)